In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.decomposition import PCA


In [ ]:
# Leer el archivo CSV
EmpleadosAttrition = pd.read_csv('empleadosRETO.csv')

# Eliminar columnas irrelevantes
EmpleadosAttrition.drop(['EmployeeCount', 'EmployeeNumber', 'Over18', 'StandardHours'], axis=1, inplace=True)


In [ ]:
# Crear columna 'Year' con el año de contratación, ignorando errores
EmpleadosAttrition['Year'] = pd.to_datetime(EmpleadosAttrition['HiringDate'], errors='coerce').dt.year

# Eliminar filas con fechas inválidas
EmpleadosAttrition.dropna(subset=['Year'], inplace=True)


In [ ]:
# Crear columna 'YearsAtCompany' calculando el tiempo hasta el año 2018
EmpleadosAttrition['YearsAtCompany'] = 2018 - EmpleadosAttrition['Year']


In [ ]:
# Limpiar la columna 'DistanceFromHome' y renombrar
EmpleadosAttrition.rename(columns={'DistanceFromHome': 'DistanceFromHome_km'}, inplace=True)
EmpleadosAttrition['DistanceFromHome'] = EmpleadosAttrition['DistanceFromHome_km'].str.replace('km', '').astype(int)

# Eliminar columnas innecesarias
EmpleadosAttrition.drop(['Year', 'HiringDate', 'DistanceFromHome_km'], axis=1, inplace=True)


In [ ]:
# Escalar la variable MonthlyIncome entre 0 y 1
scaler = MinMaxScaler()
EmpleadosAttrition['MonthlyIncome'] = scaler.fit_transform(EmpleadosAttrition[['MonthlyIncome']])


In [ ]:
# Leer nuevamente el archivo para restaurar los datos originales
EmpleadosAttrition = pd.read_csv('empleadosRETO.csv')

# Convertir variables categóricas a numéricas
label_encoders = {}
variables_categoricas = ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']

for column in variables_categoricas:
    if column in EmpleadosAttrition.columns:  # Asegurar que la columna existe antes de convertir
        le = LabelEncoder()
        EmpleadosAttrition[column] = le.fit_transform(EmpleadosAttrition[column])
        label_encoders[column] = le

# Convertir 'Attrition' y 'OverTime' explícitamente a 0 y 1
EmpleadosAttrition['Attrition'] = EmpleadosAttrition['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
EmpleadosAttrition['OverTime'] = EmpleadosAttrition['OverTime'].apply(lambda x: 1 if x == 'Yes' else 0)

# Forzar la conversión a numérica de todo el DataFrame (donde se pueda)
EmpleadosAttrition = EmpleadosAttrition.apply(pd.to_numeric, errors='coerce')

# Reemplazar NaN resultantes por 0
EmpleadosAttrition.fillna(0, inplace=True)

# Confirmar que las columnas 'Attrition' y 'OverTime' se convirtieron correctamente
print("Valores únicos en 'Attrition':", EmpleadosAttrition['Attrition'].unique())
print("Valores únicos en 'OverTime':", EmpleadosAttrition['OverTime'].unique())

# Calcular la correlación con 'Attrition'
correlaciones = EmpleadosAttrition.corr(numeric_only=True)['Attrition']
print("Correlaciones con 'Attrition':\n", correlaciones)

In [ ]:
# Realizar PCA para obtener componentes principales
pca = PCA()
EmpleadosAttritionPCA = pca.fit_transform(EmpleadosAttritionFinal.drop(['Attrition'], axis=1))

# Seleccionar componentes principales que explican al menos el 80% de la varianza
varianza_acumulada = np.cumsum(pca.explained_variance_ratio_)
componentes_necesarios = np.where(varianza_acumulada >= 0.8)[0][0] + 1


In [ ]:
# Agregar componentes principales al DataFrame final
for i in range(componentes_necesarios):
    EmpleadosAttritionFinal[f'C{i}'] = EmpleadosAttritionPCA[:, i]


In [ ]:
# Guardar el archivo final
EmpleadosAttritionFinal.to_csv('EmpleadosAttritionFinal.csv', index=False)
